In [ ]:
import pandas as pd
import seaborn as sns
# import pybettor as pb
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import math
import os
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/AML Project Fall 2022/Data Cleaning/'

Mounted at /content/gdrive


## Section 1: Import Dataset

In [ ]:
odds_og = pd.read_csv('new_odds.csv')
stats_og = pd.read_csv('total.csv')

## Section 2: Dataset Combining

#### We have scraped two datasets. Each dataset contains information for each match from the 2017-2018 season to the 2022-2023 season for the "Big-5" leagues: Premier League, Ligue 1, La Liga, Bundesliga, Serie A. This covers ~10000 matches. 

- odds: Bookmaker odds placed for each match
- stats: In-game statistics for each match

Let's look more into each dataset and join them.

In [ ]:
odds_og.head()

,COUNTRY,LEAGUE_NAME,DATE,DATE_DAY,DATE_MONTH,DATE_YEAR,TIME,MATCH_NAME,TEAM 1,TEAM 2,TABLE_SCORE,Cs,1,X,2,Bs
0,france,ligue-1,27 May 2018 - Relegation,27,May,2018,19:00,Toulouse - AC Ajaccio,Toulouse,AC Ajaccio,1:0,"['-139', '+261', '+405']",-139,+261,+405,10
1,france,ligue-1,23 May 2018 - Relegation,23,May,2018,18:45,AC Ajaccio - Toulouse,AC Ajaccio,Toulouse,0:3,"['+265', '+212', '+116']",+265,+212,+116,10
2,france,ligue-1,20 May 2018 - Relegation,20,May,2018,17:00,AC Ajaccio - Le Havre,AC Ajaccio,Le Havre,3:2 pen.,"['+125', '+204', '+244']",+125,+204,+244,10
3,france,ligue-1,19 May 2018,19,May,2018,19:00,Caen - Paris SG,Caen,Paris SG,0:0,"['+301', '+254', '-106']",+301,+254,-106,10
4,france,ligue-1,19 May 2018,19,May,2018,19:00,Dijon - Angers,Dijon,Angers,2:1,"['+129', '+259', '+200']",+129,+259,+200,10


In [ ]:
stats_og.head()

,Unnamed: 0,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Venue,Referee,Match Report,Datetime
0,0,1.0,Fri,2018-08-10,20:45 (14:45),Marseille,3.2,4–0,0.3,Toulouse,Orange Vélodrome,Ruddy Buquet,Match Report,2018-08-10
1,1,1.0,Sat,2018-08-11,17:00 (11:00),Nantes,1.4,1–3,2.6,Monaco,Stade de la Beaujoire - Louis Fonteneau,Jérôme Brisard,Match Report,2018-08-11
2,2,1.0,Sat,2018-08-11,20:00 (14:00),Nice,1.2,0–1,0.3,Reims,Stade de Nice,Johan Hamel,Match Report,2018-08-11
3,3,1.0,Sat,2018-08-11,20:00 (14:00),Angers,2.8,3–4,1.5,Nîmes,Stade Raymond Kopa,Antony Gautier,Match Report,2018-08-11
4,4,1.0,Sat,2018-08-11,20:00 (14:00),Lille,1.7,3–1,0.5,Rennes,Stade Pierre-Mauroy,Willy Delajod,Match Report,2018-08-11


In [ ]:
len(odds_og), len(stats_og), odds_og.columns

(9076,
 9250,
 Index(['COUNTRY', 'LEAGUE_NAME', 'DATE', 'DATE_DAY', 'DATE_MONTH', 'DATE_YEAR',
        'TIME', 'MATCH_NAME', 'TEAM 1', 'TEAM 2', 'TABLE_SCORE', 'Cs', '1', 'X',
        '2', 'Bs'],
       dtype='object'))

In [ ]:
odds = odds_og[~odds_og.duplicated()]
stats = stats_og[~stats_og.duplicated()] # drop duplicates

odds = odds[odds.DATE.apply(lambda row: row[-13:] != ' - Relegation')] # Remove non-season league games
odds = odds[odds['1'] != '-'] # remove cancelled games
odds['Datetime'] = pd.to_datetime(odds['DATE']) 
stats['Datetime'] = pd.to_datetime(stats['Date']) # turn dates into datetime variable

# df2['Datetime'] = df2.apply(lambda row: pd.to_datetime(str(row['DATE_DAY']) + ' ' + row['DATE_MONTH'] + ' ' + str(row['DATE_YEAR'])), axis=1)

odds['Home'] = odds['TEAM 1']
odds['Away'] = odds['TEAM 2']
odds = odds.drop(columns=['TEAM 1', 'TEAM 2'])
stats = stats.drop(columns=['Unnamed: 0']) # match team names in datasets

odds = odds[~odds.duplicated()]
stats = stats[~stats.duplicated()] # drop duplicates again

Each match can be identified by the Home team, Away team and the date of the match. Now that we've matched the columns names and types let's check that the team names are spelled the same.

In [ ]:
name_switch = {'AC Milan':'Milan', 'AS Roma':'Roma', 
                'Arminia Bielefeld':'Arminia', 'Athletic Club': 'Ath Bilbao', 
                'Atlético Madrid':'Atl. Madrid', 'B. Monchengladbach':"M'Gladbach",
                'Cardiff City':'Cardiff', 'Clermont Foot':'Clermont','Cádiz':'Cadiz', 
                'Eintracht Frankfurt':'Eint Frankfurt','Düsseldorf':'Dusseldorf', 'Granada CF': 'Granada',
                'Greuther Fürth':'Greuther Furth', 'Hannover 96':'Hannover','Hellas Verona':'Verona','Hertha Berlin':'Hertha BSC',
                'Köln':'FC Koln', 'La Coruña':'Dep. La Coruna','Leeds United':'Leeds','Leganés':'Leganes', 'Leicester City':'Leicester',
                'Mainz 05':'Mainz','Newcastle Utd':'Newcastle', 'Bayer Leverkusen':'Leverkusen', 'Norwich City':'Norwich','Alavés':'Alaves',
                'Cadiz CF':'Cadiz','Paderborn 07':'Paderborn','Nürnberg':'Nurnberg','SPAL':'Spal','Paris S-G':'Paris SG','Stoke City':'Stoke',
                'Swansea City':'Swansea','Málaga':'Malaga', 'Saint-Étienne':'St Etienne', 'Schalke 04':'Schalke','Nîmes':'Nimes'}

In [ ]:
def switch(name: str):
  if name in name_switch.keys():
    return name_switch[name]
  else: return name


In [ ]:
odds['Home'] = odds.Home.apply(lambda name: switch(name))
stats['Home'] = stats.Home.apply(lambda name: switch(name))
odds['Away'] = odds.Away.apply(lambda name: switch(name))
stats['Away'] = stats.Away.apply(lambda name: switch(name))

In [ ]:
set(odds.Home).symmetric_difference(set(stats.Home)), set(odds.Away).symmetric_difference(set(stats.Away))

({'Ajaccio', 'Almería', 'Auxerre', 'Cremonese', 'Monza', "Nott'ham Forest"},
 {'Ajaccio', 'Almería', 'Auxerre', 'Cremonese', 'Monza', "Nott'ham Forest"})

Now that we have matched the names up let's join the datasets.

In [ ]:
combined = stats.join(odds.set_index(['Datetime', 'Home', 'Away']),on=['Datetime', 'Home', 'Away'], how='inner')
# combined.to_csv(f'{path}combined_dataframe.csv')

In [ ]:
combined.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Venue,...,DATE_MONTH,DATE_YEAR,TIME,MATCH_NAME,TABLE_SCORE,Cs,1,X,2,Bs
0,1.0,Fri,2018-08-10,20:45 (14:45),Marseille,3.2,4–0,0.3,Toulouse,Orange Vélodrome,...,Aug,2018,17:45,Marseille - Toulouse,4:0,"['-222', '+343', '+651']",-222,+343,+651,10
1,1.0,Sat,2018-08-11,17:00 (11:00),Nantes,1.4,1–3,2.6,Monaco,Stade de la Beaujoire - Louis Fonteneau,...,Aug,2018,14:00,Nantes - Monaco,1:3,"['+156', '+225', '+185']",+156,+225,+185,10
2,1.0,Sat,2018-08-11,20:00 (14:00),Nice,1.2,0–1,0.3,Reims,Stade de Nice,...,Aug,2018,17:00,Nice - Reims,0:1,"['-116', '+240', '+367']",-116,+240,+367,10
3,1.0,Sat,2018-08-11,20:00 (14:00),Angers,2.8,3–4,1.5,Nimes,Stade Raymond Kopa,...,Aug,2018,17:00,Angers - Nimes,3:4,"['+129', '+219', '+238']",+129,+219,+238,10
4,1.0,Sat,2018-08-11,20:00 (14:00),Lille,1.7,3–1,0.5,Rennes,Stade Pierre-Mauroy,...,Aug,2018,17:00,Lille - Rennes,3:1,"['+171', '+219', '+174']",+171,+219,+174,10


## Section 3: Convert American Odds

The 1, X, 2 betting odds are in American odds format lets change that to a continuous probability more suitable to our model.

In [ ]:
def odds_probability(x):
    
    if x < 0:
        return  (-1*(x)) / (-1*(x) + 100)
    if x > 0:
        return  (100/(x+100))

In [ ]:
combined['1_PROB'] = combined['1'].apply(lambda x: odds_probability(int(x)))

In [ ]:
combined['X_PROB'] = combined['X'].apply(lambda x: odds_probability(int(x)))

In [ ]:
combined['2_PROB'] = combined['2'].apply(lambda x: odds_probability(int(x)))

In [ ]:
combined.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Venue', 'Referee', 'Match Report', 'Datetime', 'COUNTRY',
       'LEAGUE_NAME', 'DATE', 'DATE_DAY', 'DATE_MONTH', 'DATE_YEAR', 'TIME',
       'MATCH_NAME', 'TABLE_SCORE', 'Cs', '1', 'X', '2', 'Bs', '1_PROB',
       'X_PROB', '2_PROB'],
      dtype='object')

## Section 4: Export Dataset

In [ ]:
combined.to_csv('combined_checkpoint.csv')